In [24]:
# Comments within blocks of code start with a # sign and continue until the end of the line
import numpy as np
from math import log, sqrt, exp
from scipy import stats
from IPython.display import Image
from IPython.core.display import HTML
import pandas_datareader as pd_dr
import scipy
from scipy import *
from pandas_datareader import *
import pandas as pd
from lxml import html  
import requests
# from exceptions import ValueError
from time import sleep
import json
import argparse
from collections import OrderedDict
from time import sleep
import pandas as pd
import numpy as np

### Topic 6: Equity Valuation

#### 1. Suppose that the consensus forecast of security analysts of your favorite company is that earnings next year will be E1 = 5.00 dollars per share. Suppose that the company tends to plow back 50% of its earnings and pay the rest as dividends. If the Chief Financial Officer (CFO) estimates that the company’s growth rate will be 8% from now onwards, answer the following questions.

##### (a) If your estimate of the company’s required rate of return on its stock is 10%, what is the equilibrium price of the stock?

The equilibrium price of a stock can be determined using Gordon’s growth formula as
follows:

In [11]:
b = 0.5 # plow back ratio
R = 0.10 # required rate of return
g = 0.08 # growth rate
E1 = 5.00

In [12]:
P0 = E1*(1-b)/(R-g)
P0

124.99999999999997

##### (b) Suppose you observe that the stock is selling for 50.00 dollars per share, and that this is the best estimate of its equilibrium price. What would you conclude about either (i) your estimate of the stock’s required rate of return; or (ii) the CFO’s estimate of the company’s future growth rate?

Recall from the formula above, that a higher required rate of return implies a lower
equilibrium price of the stock. On the other hand, a higher growth rate implies a higher
stock price. Thus, a lower equilibrium stock price (50 < 125) could indicate that
either: (1) the required rate of return is higher than originally expected, and/or (2) the
growth rate is less than originally estimated.

##### (c) Suppose your own 10% estimate of the stock’s required rate of return is shared by the rest of the market. What does the market price of 50.00 dollars per share imply about the market’s estimate of the company’s growth rate?

To solve for the expected growth rate, use Gordon’s growth model:

In [13]:
P0 = 50
E1 = 5
b = 0.5
R = 0.1

In [14]:
g = R-(E1*(1-b)/P0)

In [15]:
g

0.05

#### 2. This question requires data collection. You can find all numbers on http://finance.yahoo.com.
#### The questions concern Microsoft (ticker: MSFT).

In [25]:
def parse(ticker):
	url = "http://finance.yahoo.com/quote/%s?p=%s"%(ticker,ticker)
	response = requests.get(url)
	print("Parsing ", url)
	sleep(4)
	parser = html.fromstring(response.text)
	summary_table = parser.xpath('//div[contains(@data-test,"summary-table")]//tr')
	summary_data = OrderedDict()
	other_details_json_link = "https://query2.finance.yahoo.com/v10/finance/quoteSummary/{0}?formatted=true&lang=en-US&region=US&modules=summaryProfile%2CfinancialData%2CrecommendationTrend%2CupgradeDowngradeHistory%2Cearnings%2CdefaultKeyStatistics%2CcalendarEvents&corsDomain=finance.yahoo.com".format(ticker)
	summary_json_response = requests.get(other_details_json_link)
	try:
		json_loaded_summary =  json.loads(summary_json_response.text)
		y_Target_Est = json_loaded_summary["quoteSummary"]["result"][0]["financialData"]["targetMeanPrice"]['raw']
		earnings_list = json_loaded_summary["quoteSummary"]["result"][0]["calendarEvents"]['earnings']
		eps = json_loaded_summary["quoteSummary"]["result"][0]["defaultKeyStatistics"]["trailingEps"]['raw']
		beta = json_loaded_summary["quoteSummary"]["result"][0]["defaultKeyStatistics"]["beta"]['raw']
		current_price = json_loaded_summary["quoteSummary"]["result"][0]["financialData"]["currentPrice"]['raw']
		price_to_book_ratio = json_loaded_summary["quoteSummary"]["result"][0]["defaultKeyStatistics"]["priceToBook"]['raw']
		datelist = []
		for i in earnings_list['earningsDate']:
			datelist.append(i['fmt'])
		earnings_date = ' to '.join(datelist)
		for table_data in summary_table:
			raw_table_key = table_data.xpath('.//td[@class="C(black)"]//text()')
			raw_table_value = table_data.xpath('.//td[contains(@class,"Ta(end)")]//text()')
			table_key = ''.join(raw_table_key).strip()
			table_value = ''.join(raw_table_value).strip()
			summary_data.update({table_key:table_value})
		summary_data.update({'1yr Target Est':y_Target_Est,'EPS (TTM)':eps,'Earnings Date':earnings_date,'ticker':ticker,'url':url,'Beta':beta,'Current Price':current_price,'Price/Book':price_to_book_ratio})
		return summary_data
	except ValueError:
		print("Failed to parse json response")
		return {"error":"Failed to parse json response"}
    


In [26]:
ans = parse('msft')
ans.pop('')

Parsing  http://finance.yahoo.com/quote/msft?p=msft


'91.65'

In [27]:
d = pd.DataFrame(list(ans.items()))
d1=d.T
d1.columns = d1.iloc[0]
d1=d1.reindex(d1.index.drop(0))
d1

,1yr Target Est,EPS (TTM),Earnings Date,ticker,url,Beta,Current Price,Price/Book
1,91.65,2.71,2018-01-24 to 2018-01-29,msft,http://finance.yahoo.com/quote/msft?p=msft,1.44749,82.4651,8.78036


In [28]:
import requests
from bs4 import BeautifulSoup

url = "https://finance.yahoo.com/quote/MSFT/key-statistics?p=MSFT"
url_response = requests.get(url)
soup = BeautifulSoup(url_response.text,'lxml')

#bs = soup.find_all('td')
#bs

Dividend = soup.findAll("td", {"data-reactid": "429"})[0].string
Dividend 

'1.56'

In [29]:
d1['Current Price']

1    82.4651
Name: Current Price, dtype: object

In [30]:
d1['Price to Earnings Ratio']=d1['Current Price']/d1['EPS (TTM)']
d1


,1yr Target Est,EPS (TTM),Earnings Date,ticker,url,Beta,Current Price,Price/Book,Price to Earnings Ratio
1,91.65,2.71,2018-01-24 to 2018-01-29,msft,http://finance.yahoo.com/quote/msft?p=msft,1.44749,82.4651,8.78036,30.4299


##### (b) What is the current plow-back ratio?

In [31]:
Dividend=Dividend[0:4]
Dividend=float(Dividend)
b=(d1['EPS (TTM)'].iloc[0] - Dividend)/(d1['EPS (TTM)'].iloc[0])      #b = Plowback Ratio
b

0.4243542435424354

##### (c) What is the growth rate of earnings for the next 5 years according to the analysts?

In [23]:
import requests
from bs4 import BeautifulSoup

url = "https://finance.yahoo.com/quote/MSFT/analysts?p=MSFT"
url_response = requests.get(url)
soup = BeautifulSoup(url_response.text,'lxml')

soup.find_all('td')

g = soup.findAll("td", {"data-reactid": "514"})[0].string          #g=Growth Rate
g

'10.58%'

##### (d) What is the beta of MSFT? Hint: look for "Key Statistics". If the risk-free rate (Rf ) is 5% and the market risk premium E[RM - Rf ] is 6%, what is the required rate of return on MSFT according to the CAPM?

In [24]:
B = d1['Beta']
B

1    1.44749
Name: Beta, dtype: object

In [25]:
Rf=0.05
RP=0.06
ER = Rf+B*RP
ER

1    0.136849
Name: Beta, dtype: object

##### (e) Assume that Microsoft will have earnings and dividends that will grow at the analysts forecasted rate forever after; i.e., the Gordon growth model (GGM) applies. What is the price-earnings ratio that the GGM predicts for Microsoft?

In [26]:
g=g[0:5]
g=float(g)
g=g/100

In [27]:
PE = ((1-b)*(1+g))/(ER-g)                      #Price to Earnings Ratio
PE

1    20.5012
Name: Beta, dtype: object

### Topic 7: Arbitrage

#### 3. The stock PolarBear.com trades on both the South Pole Stock Exchange and the North Pole Stock Exchange.
##### (a) Suppose the price on the North Pole is 18 dollars. What does the No-Arbitrage Condition say about the price on the South Pole? (Assume no trading costs.)

To rule out arbitrage, the price on the South Pole must be the same as on the North Pole, 18 dollars.

##### (b) Suppose the price on the North Pole is 18 dollars and the price on the the South Pole is 17 dollars? How can you make an arbitrage profit? (Assume no trading costs.)

You can make an arbitrage by buying on the the South Pole for 17 dollars and selling on the North Pole for 18 dollars. This transaction has a riskless profit of 1 dollar per share.

##### (c) Suppose that the price on the North Pole is 18 dollars, that buying or selling on the North Pole costs 2 dollars, and that buying or selling on the South Pole is free. What does the No-Arbitrage Condition say about the price on the South Pole?

The price on the South Pole must be between 16 dollars and 20 dollars. If the price is lower than 16 dollars then you can make an arbitrage profit, net of trading costs, by buying on the South Pole and selling on the North Pole. Similarly, you can make an arbitrage if the price is higher than 20 dollars.


#### 4. Suppose that there are two securities RAIN and SUN. RAIN pays 100 dollars in there is any rain during the next world cup soccer final. SUN pays 100 dollars in there is no rain. Suppose that the world cup soccer final is 1 year from today (although this is not true), and suppose that RAIN is trading at a price of 23 dollars and SUN is trading at a price of 70 dollars.

##### (a) If you buy 1 share of RAIN and 1 share of SUN, what is your payoff after 1 year, depending on the weather?

If you buy 1 share of RAIN and 1 share of SUN, then your payoff after 1 year is 100 dollars, no matter what the weather is.

##### (b) What does the No-Arbitrage Condition imply about the price of a 1-year zero-coupon bond? (Assume no trading costs.)

A portfolio of 1 share of RAIN and 1 share of SUN replicates the payoff of a zero-coupon bond, that is, it has a sure payoff of 100 dollars just like a zero. Therefore, the zero must have the same price as the portfolio, namely 93 dollars.

##### (c) Suppose that a 1-year zero-coupon bond is trading at 90 dollars. Show how you would set up a transaction to earn a riskless arbitrage profit. (Assume no trading costs.)

If the 1-year zero-coupon bond is trading at 90 dollars then you can earn a riskless arbitrage profit as follows. You buy 1 zero-coupon bond, short-sell 1 RAIN, and short-sell 1 SUN. This gives you a profit of 23+70-90= 3 dollars today, and your net cash flow next year is 0, no matter what the weather is.

##### (d) Suppose that trading zero-coupon bonds is costless, but trading RAIN and SUN each cost 2 dollars per 100 dollars face value. Can you still make an arbitrage profit?

If trading RAIN and SUN each cost 2 dollars per share then the above strategy has a trading cost of 4 dollars. This is greater than the value of the mispricing 3 dollars. Therefore, you cannot make an arbitrage profit.

### Topic 8: Fixed Income Securities

#### 5. Suppose you buy a five-year zero-coupon Treasury bond for 800 dollars per 1000 dollar face value. Answer the following questions:

##### (a) What is the yield to maturity (annual compounding) on the bond?

With F=face value, P=purchase price and t= years to maturity, the yield to maturity for a zero coupon bond is given by:

In [28]:
F = 1000
P = 800
t = 5

In [29]:
YTM = np.power((F/P),(1/t)) - 1
YTM

0.045639552591273169

##### (b) Assume the yield to maturity on comparable zeros increases to 7% immediately after purchasing the bond and remains there. Calculate your annual return (holding period yield) if you sell the bond after one year.

Annual return on a zero is the same as the holding period return and is given by:

HPR = np.power((Pt2/P),(1/t2)) - 1 , where Pt2 is the selling price of the bond and t2 is the number of years the bond is held.
This bond must also yield 7% to those you sell it to after one year. Using the formula
for the price of a zero and recalling that the 5-year bond becomes a 4-year bond after
one year, we have:

In [30]:
Pt2 = 1000/(np.power((1+0.07),4))
Pt2

762.89521204752509

Therefore, the annual return when you sell this bond after one year assuming yields
have increased to 7% is:

In [31]:
P =800
t2 = 1

HPR = np.power((Pt2/P),(1/t2)) - 1
HPR

-0.046380984940593617

Your return is less than the YTM because yields rose and you sold the bond at a lower price. Moreover, you actually lost money.

##### (c) Assume yields to maturity on comparable bonds remain at 7%, calculate your annual return if you sell the bond after two years.

The selling price after 2 years is:

In [32]:
Pt2 = 1000/(np.power((1+0.07),3))
Pt2

816.29787689085197

and your annual return over the two-year period is:

In [33]:
t2 = 2

HPR = np.power((Pt2/P),(1/t2)) - 1
HPR

0.010134815811020825

Note that even though rates are still 7%, your annual return over the two years is
positive because the selling price of the bond is now higher than after one year (as the
bond price gets pulled to par).

##### (d) Suppose after 3 years, the yield to maturity on similar zeros declines to 3%. Calculate the annual return if you sell the bond at that time.

Since the yield to maturity on comparable zeros is now 3% and there are two years left to maturity, your selling price is:

In [34]:
Pt2 = 1000/(np.power((1+0.03),2))
Pt2

942.59590913375439

and your annual return over three years is:

In [35]:
t2 = 3

HPR = np.power((Pt2/P),(1/t2)) - 1
HPR

0.05619762811510709

##### (e) If yield remains at 3%, calculate your annual return after four years.

After four years your selling price is:

In [36]:
Pt2 = 1000/(np.power((1+0.03),1))
Pt2

970.87378640776694

and your annual return over four years is:

In [37]:
t2 = 4

HPR = np.power((Pt2/P),(1/t2)) - 1
HPR

0.049586405732314764

##### (f) After five years.

After five years the bond sells for its face value because it can be redeemed for 1000 dollars.
Therefore, no matter what yields are after five years Pt2 = 1000 dollars and your annual return is:

In [38]:
Pt2 = 1000
t2 = 5

HPR = np.power((Pt2/P),(1/t2)) - 1
HPR

0.045639552591273169

##### (g) What explains the relationship between annual returns calculated in (b) through (f) and the yield to maturity in (a)?

If you sell a bond prior to maturity, the annual return earned need not equal the yield to
maturity implied by the price you paid when you purchased the bond. The calculations
in (b) and (c) show that if yields rise, your annual return is lower while (d) and (e) show
that if yields decline your annual return is higher. Nevertheless, because bond prices
are “pulled to par,” if you hold a zero coupon bond to its final maturity your annual
return will equal the yield to maturity calculation on the day you purchased the bond.

#### 6. Assume the government issues a semi-annual pay bond that matures in 5 years with a face value of $1,000 and a coupon yield of 10 percent.

##### (a) What price would you be willing to pay for such a bond if the yield to maturity (semi-annual compounding) on similar 5-year governments were 8%?

Since the yield to maturity on similar securities is 8%, you will pay a premium for a 10%
coupon bond such that the yield to maturity for both securities are equal. Since interest
payments are made semi-annually, with the face value being paid back at maturity, and
since we are using the semi-annual compounding formula, the general expression for the
price is:

We can use the annuity formula we learned earlier in the semester. Because the annuity pays twice a year, and
because compounding is semiannual, use Y TM=2 in the formula, and 2T for the number
of periods. Then:

In [40]:
t=10
R = .04
C=50
FV = 1000
P = C*((1/R)-1/(R*(1+R)**t)) + FV/((1+R)**t)
P

1081.1089577935504

##### (b) What would be the price if the yield to maturity (semi-annual compounding) on similar governments were 12%?

Because the YTM is higher than the coupon rate, the bond is sold at a discount. The
price can be computed exactly as above, using 12% instead of 8% as the YTM.

In [41]:
t=10
R = .06
C=50
FV = 1000
P = C*((1/R)-1/(R*(1+R)**t)) + FV/((1+R)**t)
P

926.3991294858529

##### (c) If the price of the bond is 103 19/32 per $100 of face value, what is the yield to maturity?

In [42]:
import scipy.optimize as optimize

def bond_ytm(price, par, T, coup, freq=2, guess=0.05):
    freq = float(freq)
    periods = T*freq
    coupon = coup/freq                     #coupon = coup/100.*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    ytm_func = lambda y: \
        sum([coupon/(1+y/freq)**(freq*t) for t in dt]) + \
        par/(1+y/freq)**(freq*5) - price
        
    return optimize.newton(ytm_func, guess)

bond_ytm(1035.90,1000,5,100)                                                   #bond_ytm(PV,FV,T,C)

0.09090578372237744

##### (d) Suppose you held the bond in (c) for 6 months, at which time you received a coupon payment and then sold the bond for a price of 102 (per $100 of face value). What would be the annualized holding period return?

In [43]:
p1 = 1035.9                                                   #Present Value
p2 = 1020                                                     #Selling Price after 6 months
C = 50
t=0.5
AnnHPR = (((p2+C)/p1)**(1/t))-1
AnnHPR

0.06692008092042245